In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from data import read_dataset
from sklearn.model_selection import train_test_split
import string
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score


## Gather data and slpit into training and testing sets

In [39]:
data = read_dataset.read_dataset('/Users/jimjr/Desktop/SoComp Assignment 2/data/pizza_request_dataset.json')
X, Y = [], []
for d in data:
    X.append(d['request_text'])
    Y.append(d['requester_received_pizza'])
xtemp = []
for s in X: #clean sentences for more accurate results
    s = s.translate(str.maketrans("", "", string.punctuation)) #remove punctiation for more accurate results
    s = s.replace('\n', ' ')
    s = s.lower()
    xtemp.append(s)
X = xtemp
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.1)
print(len(xtrain))
print(xtrain[0])


5103
basically the copays from seeing multiple doctors general physician and then a orthopedic drained my bank account and i dont have any food in the house i would greatly appreciate a helping hand


## Find top 500 unigrams and bigrams

In [40]:
ud, bd = {}, {}
for s in xtrain:
    s = s.split(' ')
    for i in range(len(s) - 1):
        ud[s[i]] = ud.get(s[i], 0) + 1
        bd[s[i] + ' ' + s[i + 1]] = bd.get(s[i] + ' ' + s[i + 1], 0) + 1
ud.pop('')
bd.pop(' ')

ud = sorted(ud.items(), key=lambda x: x[1])[-500:]
bd = sorted(bd.items(), key=lambda x: x[1])[-500:]
#print(ud)
u500, b500 = [], [] 
for t in ud:
    u500.append(t[0])
for t in bd:
    b500.append(t[0])


## Create bag of words representation using unigrams and bigrams

In [41]:
vector = CountVectorizer(ngram_range=(1,2), vocabulary=u500 + b500)
xtrain_ready = vector.fit_transform(xtrain)

## Create SVM

In [42]:
model = SVC(kernel='linear')
model.fit(xtrain_ready, ytrain)

SVC(kernel='linear')

## Test SVM

In [43]:
ypred = list(model.predict(vector.fit_transform(xtest)))
print("accuracy: " + str(accuracy_score(ytest, ypred)))
print("precision: " + str(precision_score(ytest, ypred)))
print("recall: " + str(recall_score(ytest, ypred)))
print("f1: " + str(f1_score(ytest, ypred)))
tn, fp, fn, tp = confusion_matrix(ytest, ypred).ravel()
speci = tn / (tn + fp)
print('specificity: ' + str(speci))
print("AUC: " + str(roc_auc_score(ytest, ypred)))


accuracy: 0.7165492957746479
precision: 0.42857142857142855
recall: 0.24161073825503357
f1: 0.30901287553648066
specificity: 0.8854415274463007
AUC: 0.5635261328506672
